# 01 - Binomial Pricing and Replication

Implement CRR binomial pricing, inspect replication quantities, and verify convergence to Black-Scholes.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.config import config_dict
from src.binomial import replication_tree, convergence_table_to_bs
from src.plotting import plot_binomial_convergence

cfg = config_dict(fast_mode=True)


In [ ]:
small = replication_tree(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'], N=5, option_type='call'
)
print('Root tree price:', small['price'])
print('Root replication price:', small['root_replication_price'])
print('Root replication gap:', small['root_replication_gap'])

node_rows = []
for step, vals in enumerate(small['option_values']):
    for node, val in enumerate(vals):
        node_rows.append({'step': step, 'node': node, 'option_value': float(val)})
node_df = pd.DataFrame(node_rows)

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(node_df['step'], node_df['option_value'], alpha=0.8)
ax.set_title('Option Values Across Nodes (N=5)')
ax.set_xlabel('Step')
ax.set_ylabel('Option Value')
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'binomial_nodes_n5.png', dpi=150)
plt.close(fig)

delta_rows = []
for step, vals in enumerate(small['deltas']):
    for node, val in enumerate(vals):
        delta_rows.append({'step': step, 'node': node, 'delta': float(val)})
delta_df = pd.DataFrame(delta_rows)

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(delta_df['step'], delta_df['delta'], alpha=0.8)
ax.set_title('Deltas Across Nodes (N=5)')
ax.set_xlabel('Step')
ax.set_ylabel('Delta')
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'binomial_deltas_n5.png', dpi=150)
plt.close(fig)


In [ ]:
N_values = [10, 25, 50, 100, 200, 400]
conv = convergence_table_to_bs(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'], option_type='call', N_values=N_values
)
conv.to_csv(ROOT / 'results' / 'tables' / 'binomial_convergence.csv', index=False)
plot_binomial_convergence(conv, ROOT / 'results' / 'figures' / 'binomial_convergence.png')
conv
